In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from pathlib import Path

from dfm_models.utils.io import download_COOPs, download_nwis
from dfm_models.utils.openDA import writeNoosTs, createNoosConfigFile

In [ ]:
project = Path("/mnt/c/Users/rdchlclj/Projects/MR_D3D_model")
stations = pd.read_excel(project / "Delft3D/observations/stations.xlsx", dtype={"station_id": str})

In [ ]:
output = Path.cwd() / "dumdumstochObserver"

product = "hourly_height"
datum = "MSL"
form = "csv"
begin_date = "20160201"
end_date = "20160301"
unit = "waterlevel"

dt = pd.to_timedelta(6, "H")

t0 = pd.to_datetime("2016-02-01 00:00")
tf = pd.to_datetime("2016-03-01 00:00")

AOIStations = {
    "Dauphin_Island": 0.5,
    "Pascagoula": 0.5,
    "Bay_Waveland_Yacht_Club": 0.5,
    "Shell_Beach": 0.25,
    "New_Canal": 0.25,
    "Bonnet_Carre_Spillway": 0.25
}

baseStdev = 0.05

useStdev = []
useStations = []

In [ ]:
# for i, row in stations[stations.obsFile_id.isin(useStations)].iterrows():
for i, row in stations.iterrows():

    station_id = row.station_id
    obsFile_id = row.obsFile_id
    x = row.longitude
    y = row.latitude
    provider = row.provider

    if provider == "NOAA":
        data = download_COOPs(
            product, obsFile_id, station_id, datum, begin_date, end_date
        )

        try:
            stdev = AOIStations[obsFile_id] * baseStdev
        except KeyError:
            stdev = baseStdev

        # data.index += dta
        data = data.loc[t0:tf]

        if data.isna().any():
            print(obsFile_id)
            continue

        useStations.append(obsFile_id)
        useStdev.append(stdev)

        fn = output / f"{obsFile_id}"
        writeNoosTs(data, obsFile_id, x, y, unit, fn)

fn = output / "noosObservations.xml"
createNoosConfigFile(useStdev, useStations, fn)